### Merge Test HRRRAK Data

Notebook contents 
* from script `./download_hrrrak_final.sh 2025 03 01` 
* the goal is to look at all the variables and figure out how to manage them 

created by Cassie Lumbrazo\
last updated: March 2025\
run location: UAS linux\
python environment: **xarray**

In [2]:
# import packages 
%matplotlib inline

# plotting packages 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 

sns.set_theme()
# plt.rcParams['figure.figsize'] = [12,6] #overriding size

# data packages 
import pandas as pd
import numpy as np
import xarray as xr
from datetime import datetime

import scipy

In [3]:
from matplotlib.patches import Patch
from matplotlib.colors import ListedColormap
from matplotlib import ticker

In [4]:
pwd

'/home/cassie/python/repos/juneau_data_analysis'

The entire list of variables we should have are, 
* pressure, PRES, SP, surface 
* orography, HGT, OROG, surface
* air temperature, TMP surface, 
* relative humidity, RH 2m
* U-component wind, UGRD 10m, **not importing with the ds...*
* V-component wind, VGRD 10m, **not importing with the ds...*
* in shortwave rad, DWSRF surface 
* in longwave rad, DLWRF surface
* TMP, temperature, 2m 
* precipitation, APCP surface, **not importing with the ds...*
* precipitation, PRATE surface
* snow depth, SNOD surface
* ground temperature, TSOIL 0-0m, **this is not downloading at all as of right now**
* SWE, WEASD surface
* snow cover, SNOWC surface
* snowfall, ASNOW surface **this is not downloading at all as of right now**

In [5]:
import cfgrib

In [19]:
filepath = '/home/cassie/python/download_hrrr/'
# file = 'hrrr.20250301/hrrr.t00z.wrfsfcf00.ak.grib2' # why not forcast 0? 
filename = 'hrrr.20250301/hrrr.t00z.wrfsfcf01.ak.grib2'
# filename = 'hrrr.20250301/hrrr.t00z.wrfsfcf02.ak.grib2'
# filename = 'hrrr.20250301/hrrr.t00z.wrfsfcf03.ak.grib2'

ds = cfgrib.open_datasets(filepath + filename)

# loop through for all ds
ds_list = []
for i in range(len(ds)):
    ds_list.append(ds[i])
    
ds_10m = ds_list[0].copy()
ds_2m = ds_list[1].copy()
ds_0m = ds_list[2].copy()
ds_0m = ds_0m.assign_coords({'heightAboveGround': 0.0}) 

## Merge the datasets to get all variables together

In [20]:
# drop the heightAboveGround coordinate for all three datasets 
ds_10m = ds_10m.drop('heightAboveGround')
ds_2m = ds_2m.drop('heightAboveGround')
ds_0m = ds_0m.drop('heightAboveGround')

# now merge all the datasets from ds_list...
ds = xr.merge([ds_0m, ds_2m, ds_10m]) # but height above ground is still an issue since they have different values... 
ds

/tmp/ipykernel_3521832/3283579432.py:2: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds_10m = ds_10m.drop('heightAboveGround')
/tmp/ipykernel_3521832/3283579432.py:3: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds_2m = ds_2m.drop('heightAboveGround')
/tmp/ipykernel_3521832/3283579432.py:4: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds_0m = ds_0m.drop('heightAboveGround')


<xarray.Dataset> Size: 32kB
Dimensions:     (y: 20, x: 21)
Coordinates:
    time        datetime64[ns] 8B 2025-03-01
    step        timedelta64[ns] 8B 01:00:00
    surface     float64 8B 0.0
    latitude    (y, x) float64 3kB 58.08 58.08 58.08 58.08 ... 58.59 58.59 58.59
    longitude   (y, x) float64 3kB 225.1 225.1 225.2 225.2 ... 226.0 226.0 226.1
    valid_time  datetime64[ns] 8B 2025-03-01T01:00:00
Dimensions without coordinates: y, x
Data variables: (12/15)
    unknown     (y, x) float32 2kB ...
    t           (y, x) float32 2kB ...
    sp          (y, x) float32 2kB ...
    prate       (y, x) float32 2kB ...
    sde         (y, x) float32 2kB ...
    orog        (y, x) float32 2kB ...
    ...          ...
    sdswrf      (y, x) float32 2kB ...
    sdlwrf      (y, x) float32 2kB ...
    t2m         (y, x) float32 2kB ...
    r2          (y, x) float32 2kB ...
    u10         (y, x) float32 2kB ...
    v10         (y, x) float32 2kB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP

And, get wind calculated with attributes

In [21]:
# calculate windspeed from u and v
ds['wind'] = np.sqrt(ds['u10']**2 + ds['v10']**2)
ds['wind'].attrs =  ds['u10'].attrs

ds['wind'].attrs['long_name'] = '10 metre wind speed calculated from u and v wind components'
ds['wind'].attrs['GRIB_shortName'] = '10m wind'
ds['wind'].attrs['standard_name'] = 'wind speed'
ds['wind'].attrs['GRIB_name'] = '10 metre wind speed'
ds['wind'].attrs['GRIB_cfName'] = 'wind_speed'
ds['wind'].attrs['GRIB_cfVarName'] ='wind10'
ds['wind'].attrs

{'GRIB_paramId': 165,
 'GRIB_dataType': 'fc',
 'GRIB_numberOfPoints': 420,
 'GRIB_typeOfLevel': 'heightAboveGround',
 'GRIB_stepUnits': 1,
 'GRIB_stepType': 'instant',
 'GRIB_gridType': 'polar_stereographic',
 'GRIB_NV': 0,
 'GRIB_cfName': 'wind_speed',
 'GRIB_cfVarName': 'wind10',
 'GRIB_gridDefinitionDescription': 'Polar stereographic can be south or north',
 'GRIB_missingValue': 3.4028234663852886e+38,
 'GRIB_name': '10 metre wind speed',
 'GRIB_shortName': '10m wind',
 'GRIB_units': 'm s**-1',
 'long_name': '10 metre wind speed calculated from u and v wind components',
 'units': 'm s**-1',
 'standard_name': 'wind speed'}

In [22]:
# now, let's rename a few things to make this easier 
# ds = ds.rename({'t':   'temp_0m',
#                 'sp':  'pres_0m', 
#                 't2m': 'temp_2m',
#                 'r2':  'rh_2m', 
#                 'sdwe':'swe', 
#                 'sde': 'hs',
#                 'sdswrf': 'swrad',
#                 'sdlwrf': 'lwrad',
#                 'tp': 'precip_accum',
#                 'prate': 'precip_rate',
#                 })

ds = ds.rename({'t':   'temp_surface',
                'sp':  'pressure', 
                't2m': 'temp',
                'r2':  'rh', 
                'sdwe':'swe', 
                'sde': 'snowdepth',
                'sdswrf': 'swrad',
                'sdlwrf': 'lwrad',
                'tp': 'precip_total',
                'prate': 'precip_rate',
                })

ds = ds.drop_vars('unknown')

ds

<xarray.Dataset> Size: 32kB
Dimensions:       (y: 20, x: 21)
Coordinates:
    time          datetime64[ns] 8B 2025-03-01
    step          timedelta64[ns] 8B 01:00:00
    surface       float64 8B 0.0
    latitude      (y, x) float64 3kB 58.08 58.08 58.08 ... 58.59 58.59 58.59
    longitude     (y, x) float64 3kB 225.1 225.1 225.2 ... 226.0 226.0 226.1
    valid_time    datetime64[ns] 8B 2025-03-01T01:00:00
Dimensions without coordinates: y, x
Data variables: (12/15)
    temp_surface  (y, x) float32 2kB ...
    pressure      (y, x) float32 2kB ...
    precip_rate   (y, x) float32 2kB ...
    snowdepth     (y, x) float32 2kB ...
    orog          (y, x) float32 2kB ...
    precip_total  (y, x) float32 2kB ...
    ...            ...
    lwrad         (y, x) float32 2kB ...
    temp          (y, x) float32 2kB ...
    rh            (y, x) float32 2kB ...
    u10           (y, x) float32 2kB -3.468 -1.343 -0.2181 ... 1.219 3.469
    v10           (y, x) float32 2kB 6.355 4.043 1.043 ... 2.793 0.6676 -0.2074
    wind          (y, x) float32 2kB 7.24 4.26 1.065 0.596 ... 2.936 1.39 3.476
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP

Now, it's time to try to merge multiple datetimes together... 